In [3]:
#!pip3 install line_profiler

In [1]:
import meshio
import numpy as np
from multiprocessing import Pool
import time
import functools
from queue import PriorityQueue

#from ugs import *
# Need to make sure it's installed: pip3 install line_profiler
%load_ext line_profiler

## Get paleo-topographies

In [2]:
# Specify most recent time in Ma
startMa = 0

# Specify deepest time in Ma
endMa = 100

# Specify paleomap interval
dtMa = 5

timeframe = np.arange(startMa,endMa+dtMa,dtMa)

In [3]:
for k in range(len(timeframe)):
    paleofile = 'earth/elev_npz/elev'+str(timeframe[k])+'Ma.npz'
    paleomesh = 'earth/elev_vtk/elev'+str(timeframe[k])+'Ma.vtk'

    topo = np.load(paleofile)
    cells = topo['c']
    vertices = topo['v']
    elev = topo['z']

    vtk_mesh = meshio.Mesh(vertices, {'triangle': cells}, point_data={"Z":elev})
    meshio.write(paleomesh, vtk_mesh)

    print("Writing VTK file {}".format(paleomesh))

Writing VTK file earth/elev_vtk/elev0Ma.vtk
Writing VTK file earth/elev_vtk/elev5Ma.vtk
Writing VTK file earth/elev_vtk/elev10Ma.vtk
Writing VTK file earth/elev_vtk/elev15Ma.vtk
Writing VTK file earth/elev_vtk/elev20Ma.vtk
Writing VTK file earth/elev_vtk/elev25Ma.vtk
Writing VTK file earth/elev_vtk/elev30Ma.vtk
Writing VTK file earth/elev_vtk/elev35Ma.vtk
Writing VTK file earth/elev_vtk/elev40Ma.vtk
Writing VTK file earth/elev_vtk/elev45Ma.vtk
Writing VTK file earth/elev_vtk/elev50Ma.vtk
Writing VTK file earth/elev_vtk/elev55Ma.vtk
Writing VTK file earth/elev_vtk/elev60Ma.vtk
Writing VTK file earth/elev_vtk/elev65Ma.vtk
Writing VTK file earth/elev_vtk/elev70Ma.vtk
Writing VTK file earth/elev_vtk/elev75Ma.vtk
Writing VTK file earth/elev_vtk/elev80Ma.vtk
Writing VTK file earth/elev_vtk/elev85Ma.vtk
Writing VTK file earth/elev_vtk/elev90Ma.vtk
Writing VTK file earth/elev_vtk/elev95Ma.vtk
Writing VTK file earth/elev_vtk/elev100Ma.vtk


In [48]:
@functools.lru_cache(maxsize=32768)
def distance(current, _next):
    # from https://stackoverflow.com/a/1401828
    if current == _next:
        return 0
    return int(np.linalg.norm(mesh.points[current]-mesh.points[_next]))

@functools.lru_cache(maxsize=183746)
def graph_neighbours(current):
    # Get the points from the cells which have the same point in them
    points = np.unique(mesh.cells_dict['triangle'][np.where(mesh.cells_dict['triangle']==current)[0]])
    # remove the current point from these results:
    points = points[points != current]
    
    # Get the elevation of those connected points
    elevations = mesh.point_data['Z'][points]
    # Return a list of connected points, as long as they are above sea-level
    return points[elevations >= 0]
    

def cost_search(start, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    frontier = PriorityQueue()  # The priority queue means that we can find the least cost path to continue
    frontier.put(start, 0)      # from, along any path, meaning the resulting paths should always be the least-cost
                                # path to get to that point.
    came_from = {}
    cost_so_far = {}
    dist_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    dist_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        for _next in graph_neighbours(current):
            # Calculate the cost of going to this new point.
            new_cost = cost_so_far[current] + travel_cost_function(current, _next)
            # Calculate the eulerian distance to this new point.
            new_dist = dist_so_far[current] + distance(current, _next)

            # The max_distance check tells the algorithm to stop once we start getting too far away from the starting point.
            if (_next not in cost_so_far or new_cost < cost_so_far[_next]) and new_dist < max_distance:
                cost_so_far[_next] = new_cost
                dist_so_far[_next] = new_dist
                priority = new_cost
                frontier.put(_next, priority)
                came_from[_next] = current

        
    return came_from, cost_so_far

def cost_search(start, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    frontier = PriorityQueue()  # The priority queue means that we can find the least cost path to continue
    frontier.put(start, 0)      # from, along any path, meaning the resulting paths should always be the least-cost
                                # path to get to that point.
    came_from = {}
    cost_so_far = {}
    dist_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    dist_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        for _next in graph_neighbours(current):
            # Calculate the cost of going to this new point.
            new_cost = cost_so_far[current] + travel_cost_function(current, _next)
            # Calculate the eulerian distance to this new point.
            new_dist = dist_so_far[current] + distance(current, _next)

            # The max_distance check tells the algorithm to stop once we start getting too far away from the starting point.
            if (_next not in cost_so_far or new_cost < cost_so_far[_next]) and new_cost <= max_fuel:
                cost_so_far[_next] = new_cost
                dist_so_far[_next] = new_dist
                priority = new_cost
                frontier.put(_next, priority)
                came_from[_next] = current

        
    return came_from, cost_so_far, dist_so_far

def get_total_cost_for_point(start, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    came_from, cost_so_far, dist_so_far = cost_search(start, travel_cost_function, max_distance, graph_neighbours=graph_neighbours)
    
    # Find the edge nodes, and add up their costs to get the total
    total_cost = 0
    for k in came_from.keys():             # For all the points we've visited,
        if k not in came_from.values():    # Find all the points that haven't been 'came_from'
            total_cost += cost_so_far[k]
            
    return total_cost

def get_total_distance_for_all_paths_to_point(start, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    came_from, cost_so_far, dist_so_far = cost_search(start, travel_cost_function, max_fuel, graph_neighbours=graph_neighbours)
    
    # Find the edge nodes, and add up their costs to get the total
    total_dist = 0
    for k in came_from.keys():             # For all the points we've visited,
        if k not in came_from.values():    # Find all the points that haven't been 'came_from'
            total_dist += dist_so_far[k]
            
    return total_dist

def get_from_cell(cell, travel_cost_function, max_distance=100000):
    start = cell[1]
    return get_from_point(start, travel_cost_function, max_distance)

def get_dist_from_point(point, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    # Return a tuple of (the point id, it's cost)
    total_dist = get_total_distance_for_all_paths_to_point(point, travel_cost_function, max_fuel, graph_neighbours=graph_neighbours)
    #print(os.getpid(), distance.cache_info())
    return (point, total_dist)

def get_from_point(point, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    # Return a tuple of (the point id, it's cost)
    total_cost = get_total_cost_for_point(point, travel_cost_function, max_distance, graph_neighbours)
    #print(os.getpid(), distance.cache_info())
    return (point, total_cost)

In [49]:
# We need to define a way to calculate cost
@functools.lru_cache(maxsize=1048576)
def distance_with_elevation_scaled(current, _next):
    # The travel_cost can be any function, including just the distance.
    # Here, we exagerate the elevation difference, to make changing elevation more costly
    if current == _next:
        return 0
    
    z_scaling = 100.  # 100. is a random number to pick, but has quite a big impact on the resulting paths.
    
    new_current = np.append(mesh.points[current][:2], mesh.point_data['Z'][current] * z_scaling)
    new_next    = np.append(mesh.points[_next][:2],   mesh.point_data['Z'][_next]   * z_scaling)
    
    return int(np.linalg.norm(new_current - new_next))  # return as Int, just for niceness

In [50]:
# We need to define a way to calculate cost
@functools.lru_cache(maxsize=1048576)
def elevation_only(current, _next):
    # Only take into account elevation changes for costs
    if current == _next:
        return 0
    return int(abs(mesh.point_data['Z'][current] - mesh.point_data['Z'][_next]) + distance(current, _next)*0.004)

In [51]:
k = -1
paleomesh = 'earth/elev_vtk/elev'+str(timeframe[k])+'Ma.vtk'
paleobio = 'earth/bio_vtk/elev'+str(timeframe[k])+'Ma.vtk'

mesh = meshio.read(paleomesh)

max_fuel = 2500
# A smaller value means visiting far fewer nodes, so it speeds things up a lot

In [52]:
# Choose which cost function you want
travel_cost = elevation_only

## Prepare input data

We don't want to calculate the LEC of a point below sea-level, so here we find all the points above sea-level, so they can be used as starting points.

In [53]:
points_above_sealevel = np.nonzero(mesh.point_data['Z'] >= 0)[0][::-1]

In [54]:
print("Total starting points available: ", points_above_sealevel.shape[0])

Total starting points available:  179477


## Prepare for parallel execution

We want to use a parallel map function to calculate the cost for all points. We can make this easier by 'baking in' the parameters of a function we know, and leave only the `starting point` as a variable.

In [55]:
from functools import partial

# We bake in the mesh and travel_cost function into the get_from_point function
get_from_point_in_mesh = partial(get_dist_from_point, travel_cost_function = travel_cost, max_fuel = max_fuel)

In [56]:
#%lprun -f get_from_point_in_mesh get_from_point_in_mesh(points_above_sealevel[100])

In [57]:
# Now we can use the get_from_point function with only a point ID as a parameter, by going via the new get_from_point_in_mesh function
# Here we do a test, and see the output format: (point, total cost of all paths to that point)
print(travel_cost(points_above_sealevel[100], points_above_sealevel[101]))
print(get_from_point_in_mesh(points_above_sealevel[100]))
print(graph_neighbours.cache_info())

759
(655193, 8836398)
CacheInfo(hits=65, misses=103, maxsize=183746, currsize=103)


In [58]:
# Output data variables
all_costs = []
mesh.point_data['cost'] = np.zeros_like(mesh.point_data['Z'])

p = Pool(6)
start = 0
inc = 1000
stop = inc

while start < points_above_sealevel.shape[0]:
    start_time = time.time()
    # Here we use a parallel map function to send out the chunk across the CPUs in the Pool
    costs = p.map(get_from_point_in_mesh, points_above_sealevel[start:stop])
    print("From ", start, " to ", stop, " took ", time.time() - start_time, "seconds, starting with point", points_above_sealevel[start], ". Percent complete: ", 100*(float(stop)/points_above_sealevel.shape[0]))
    # Save the data
    all_costs.extend(costs)
    
    # Write out data progressively, so we can see progress in Paraview
    for i in all_costs:
        mesh.point_data['cost'][i[0]] = i[1]
    meshio.write(paleobio, mesh)
    
    # move to the next chunk of data
    start += inc
    stop += inc
    if stop >= points_above_sealevel.shape[0]:
        stop = points_above_sealevel.shape[0]-1

From  0  to  1000  took  259.7386245727539 seconds, starting with point 655361 . Percent complete:  0.5571744568941981
From  1000  to  2000  took  146.82976126670837 seconds, starting with point 648565 . Percent complete:  1.1143489137883962
From  2000  to  3000  took  233.93369841575623 seconds, starting with point 640790 . Percent complete:  1.6715233706825945
From  3000  to  4000  took  227.4308364391327 seconds, starting with point 636079 . Percent complete:  2.2286978275767924
From  4000  to  5000  took  126.67887139320374 seconds, starting with point 632562 . Percent complete:  2.7858722844709907
From  5000  to  6000  took  140.7515630722046 seconds, starting with point 631379 . Percent complete:  3.343046741365189
From  6000  to  7000  took  237.72814226150513 seconds, starting with point 629869 . Percent complete:  3.900221198259387
From  7000  to  8000  took  183.53801822662354 seconds, starting with point 626342 . Percent complete:  4.457395655153585
From  8000  to  9000  too

From  67000  to  68000  took  21.66580033302307 seconds, starting with point 410165 . Percent complete:  37.88786306880547
From  68000  to  69000  took  14.271286487579346 seconds, starting with point 406932 . Percent complete:  38.44503752569967
From  69000  to  70000  took  15.931782245635986 seconds, starting with point 405101 . Percent complete:  39.00221198259387
From  70000  to  71000  took  16.556103944778442 seconds, starting with point 400648 . Percent complete:  39.55938643948807
From  71000  to  72000  took  14.18777585029602 seconds, starting with point 396737 . Percent complete:  40.11656089638226
From  72000  to  73000  took  19.014466524124146 seconds, starting with point 391665 . Percent complete:  40.67373535327646
From  73000  to  74000  took  17.581751823425293 seconds, starting with point 389265 . Percent complete:  41.230909810170665
From  74000  to  75000  took  12.302581548690796 seconds, starting with point 386149 . Percent complete:  41.78808426706486
From  750

From  133000  to  134000  took  14.10029673576355 seconds, starting with point 169070 . Percent complete:  74.66137722382256
From  134000  to  135000  took  15.183168411254883 seconds, starting with point 166055 . Percent complete:  75.21855168071674
From  135000  to  136000  took  12.220017671585083 seconds, starting with point 160742 . Percent complete:  75.77572613761095
From  136000  to  137000  took  14.206059694290161 seconds, starting with point 154745 . Percent complete:  76.33290059450515
From  137000  to  138000  took  13.564538717269897 seconds, starting with point 152500 . Percent complete:  76.89007505139934
From  138000  to  139000  took  13.440089464187622 seconds, starting with point 150059 . Percent complete:  77.44724950829354
From  139000  to  140000  took  13.068712711334229 seconds, starting with point 147891 . Percent complete:  78.00442396518774
From  140000  to  141000  took  15.641607522964478 seconds, starting with point 142857 . Percent complete:  78.56159842

## Output

Show some of the results:

In [43]:
# print(all_costs[:100])